In [1]:
import pandas as pd
import numpy as np

print('Версия pandas:', pd.__version__)

Версия pandas: 2.1.4


# Иерархический индекс. Многоуровневый индекс MultiIndex

    План урока

    Часть 1. Создание объекта MultiIndex
    
    1.1 Автоматическое создание иерархического индекса во время группировки и агрегации данных
    
    1.2 Различные способы инициализации MultiIndex
    
    - Метод .from_arrays() - из массивов
    - Метод .from_frame() - из объекта DataFrame
    - Метод .from_product() - Прямое (Декартово) произведение
    - Метод .from_tuples() - из списка кортежей
    - с помощью аргументов levels и codes
    - передача списка массивов напрямую в Series или DataFrame
    
    Часть 2.
    
    2.1 Свойства и методы атрибутов index и columns
    
    - index.nlevels - Узнаем количество уровней
    - index.levshape - Узнаем длину каждого уровня
    - index.names -  Узнаем названия уровней
    - Метод index.set_names() - Смена имён уровней
    - Метод index.get_level_values() - Просмотр меток уровней MultiIndex
    - index.levels - Получаем уникальные элементы, содержащиеся в MultiIndex
    - Передача нового мультииндекса через атрибут index или columns
    - Метод index.set_levels() - Изменение значений индексов на конкретных уровнях
    - Метод index.remove_unused_levels() - Удаление неиспользуемого уровня
    - Метод index.droplevel() - Удаление конкретного уровня
    
    2.2 Методы Series и DataFrame для работы с Index и MultiIndex
    
    - DataFrame.set_index() - Установка индекса (MultiIndex)
    - DataFrame.reset_index() - Сброс индекса (уровней MultiIndex)
    - DataFrame.swaplevel() - Смена уровней
    - DataFrame.reorder_levels() - Изменение порядка уровней 
    - DataFrame.reindex() -  Использование переиндексации MultiIndex
    - DataFrame.rename() - Переименование меток Index или MultiIndex
    - DataFrame.rename_axis() - Переименование уровней MultiIndex (имени Index)
    - DataFrame.sort_index() - Сортировка уровней MultiIndex (Index)
    - другие свойства Index (MultiIndex)

    Часть 3. Доступ к данным через MultiIndex
    
    - Метод .loc[ ]
    - Доступ к данным без специальных методов
    - Метод .xs( ) cross section
    - Метод take( )
    - Метод slice ( )

    Объект MultiIndex является иерархическим аналогом стандартного объекта Index, который хранит метки осей. 

## Часть 1. Создание объекта MultiIndex

## 1.1 Автоматическое создание иерархического индекса во время группировки и агрегации данных

In [2]:
df = pd.DataFrame({'key1': ['A', 'B', 'C', 'A', 'B', 'C','B', 'B', 'C', 'C', 'B', 'C'],
                   'key2': ['yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes'],
                   'data1': range(12),
                   'data2': np.random.randint(0,10, 12)})
df

,key1,key2,data1,data2
0,A,yes,0,8
1,B,yes,1,2
2,C,no,2,4
3,A,no,3,9
4,B,no,4,2
5,C,yes,5,9
6,B,no,6,9
7,B,yes,7,1
8,C,no,8,2
9,C,no,9,5


In [3]:
df.index

RangeIndex(start=0, stop=12, step=1)

In [4]:
df.columns

Index(['key1', 'key2', 'data1', 'data2'], dtype='object')

In [5]:
display(df)
print(df)

,key1,key2,data1,data2
0,A,yes,0,8
1,B,yes,1,2
2,C,no,2,4
3,A,no,3,9
4,B,no,4,2
5,C,yes,5,9
6,B,no,6,9
7,B,yes,7,1
8,C,no,8,2
9,C,no,9,5


   key1 key2  data1  data2
0     A  yes      0      8
1     B  yes      1      2
2     C   no      2      4
3     A   no      3      9
4     B   no      4      2
5     C  yes      5      9
6     B   no      6      9
7     B  yes      7      1
8     C   no      8      2
9     C   no      9      5
10    B   no     10      8
11    C  yes     11      5


In [6]:
# создание объекта DataFrameGroupBy
df.groupby(['key1'])

#### Пример 1.1

In [7]:
# выбираем группирующий столбец ('key1')
# выбираем столбцы для агрегации (['data1','data2'])
# используем метод agg(), выбираем несколько агрегирующих функций (['sum', 'mean'])

df.groupby('key1')[['data1','data2']].agg(['sum', 'mean'])

data1      data2     
       sum mean   sum mean
key1                      
A        3  1.5    17  8.5
B       28  5.6    22  4.4
C       35  7.0    25  5.0

#### Пример 1.2

In [8]:
# выбираем несколько группирующих столбцов (['key1','key2'])
# по умолчанию агрегация производится по всем остальным столбцам ('data1','data2')
# выбираем агрегирующую функцию sum()

df.groupby(['key1','key2']).sum()

data1  data2
key1 key2              
A    no        3      9
     yes       0      8
B    no       20     19
     yes       8      3
C    no       19     11
     yes      16     14

#### Пример 1.3

In [9]:
agg_result=df.groupby(['key1','key2'])[['data1','data2']].agg(['sum', 'mean']).round(2)
agg_result

data1       data2      
            sum  mean   sum  mean
key1 key2                        
A    no       3  3.00     9  9.00
     yes      0  0.00     8  8.00
B    no      20  6.67    19  6.33
     yes      8  4.00     3  1.50
C    no      19  6.33    11  3.67
     yes     16  8.00    14  7.00

In [10]:
agg_result.index

MultiIndex([('A',  'no'),
            ('A', 'yes'),
            ('B',  'no'),
            ('B', 'yes'),
            ('C',  'no'),
            ('C', 'yes')],
           names=['key1', 'key2'])

In [11]:
agg_result.columns

MultiIndex([('data1',  'sum'),
            ('data1', 'mean'),
            ('data2',  'sum'),
            ('data2', 'mean')],
           )

#### Пример 1.4.1

In [12]:
# сохраняем многоуровневый DataFrame в файл csv

agg_result.to_csv('files/agg_result')

In [13]:
# открываем сохраненный csv-файл 

new_agg_res=pd.read_csv('files/agg_result')
new_agg_res

,Unnamed: 0,Unnamed: 1,data1,data1.1,data2,data2.1
0,NaN,NaN,sum,mean,sum,mean
1,key1,key2,NaN,NaN,NaN,NaN
2,A,no,3,3.0,9,9.0
3,A,yes,0,0.0,8,8.0
4,B,no,20,6.67,19,6.33
5,B,yes,8,4.0,3,1.5
6,C,no,19,6.33,11,3.67
7,C,yes,16,8.0,14,7.0


In [14]:
# просматриваем одновременно индексы строк и столбцов
new_agg_res.axes

[RangeIndex(start=0, stop=8, step=1),
 Index(['Unnamed: 0', 'Unnamed: 1', 'data1', 'data1.1', 'data2', 'data2.1'], dtype='object')]

#### Пример 1.4.2

In [15]:
new_agg_res=pd.read_csv('files/agg_result', index_col=[0,1], header=[0,1])
new_agg_res

data1       data2      
            sum  mean   sum  mean
key1 key2                        
A    no       3  3.00     9  9.00
     yes      0  0.00     8  8.00
B    no      20  6.67    19  6.33
     yes      8  4.00     3  1.50
C    no      19  6.33    11  3.67
     yes     16  8.00    14  7.00

## 1.2 Различные способы инициализации MultiIndex

    MultiIndex - это индекс, состоящий из нескольких уровней
    Каждый уровень (level) такого индекса может обладать собственным именем (name)
    Все конструкторы MultiIndex принимают аргумент names, который хранит имена/метки (в виде строки) для самих уровней

## Пример № 1.5
## Метод .from_arrays() - из массивов

In [16]:
arrays=[['group_1','group_1','group_2','group_2'],['A','B','C','D']]
multi_ind_arrays= pd.MultiIndex.from_arrays(arrays, names=['level_0', 'level_1'])
multi_ind_arrays

MultiIndex([('group_1', 'A'),
            ('group_1', 'B'),
            ('group_2', 'C'),
            ('group_2', 'D')],
           names=['level_0', 'level_1'])

In [17]:
# создаем рандомную матрицу из целых чисел 4x4

random_matrix = np.random.randint(0,100, size=(4, 4))
random_matrix

array([[51, 78, 71, 16],
       [67, 43, 81, 57],
       [79, 42, 67, 20],
       [90, 29, 91, 52]])

#### Многоуровневый индекс строк
    Пример № 1.5.1

In [18]:
df_1_index = pd.DataFrame(random_matrix, index=multi_ind_arrays)
df_1_index

0   1   2   3
level_0 level_1                
group_1 A        51  78  71  16
        B        67  43  81  57
group_2 C        79  42  67  20
        D        90  29  91  52

#### Многоуровневый индекс столбцов
    Пример № 1.5.2

In [19]:
df_2_columns = pd.DataFrame(random_matrix, columns=multi_ind_arrays)
df_2_columns

level_0 group_1     group_2    
level_1       A   B       C   D
0            51  78      71  16
1            67  43      81  57
2            79  42      67  20
3            90  29      91  52

#### Многоуровневый индекс строк и столбцов
    Пример № 1.5.3

In [20]:
df_3_index_columns = pd.DataFrame(random_matrix, index=multi_ind_arrays, columns=multi_ind_arrays)
df_3_index_columns

level_0         group_1     group_2    
level_1               A   B       C   D
level_0 level_1                        
group_1 A            51  78      71  16
        B            67  43      81  57
group_2 C            79  42      67  20
        D            90  29      91  52

#### Многоуровневый индекс строк и столбцов (без названий уровней)
    Пример № 1.5.4

In [21]:
# MultiIndex без названий уровней

multi_ind_arrays2= pd.MultiIndex.from_arrays(arrays)
pd.DataFrame(random_matrix, index=multi_ind_arrays2, columns=multi_ind_arrays2)

group_1     group_2    
                A   B       C   D
group_1 A      51  78      71  16
        B      67  43      81  57
group_2 C      79  42      67  20
        D      90  29      91  52

In [22]:
# создаем 2 рандомных массива из 6-ти целых чисел

random_array1 = np.random.randint(0, 100, size=(6))
random_array2 = np.random.randint(0, 100, size=(6))

## Пример № 1.6
## Метод .from_frame() - из объекта DataFrame

In [23]:
df_mult_ind = pd.DataFrame([['A', 'one'], ['A', 'two'], ['B', 'one'], ['B', 'two'],['C', 'one'], ['C', 'two']],
                  columns=['first', 'second'])
display(df_mult_ind)

multi_ind_frame=pd.MultiIndex.from_frame(df_mult_ind)
multi_ind_frame

,first,second
0,A,one
1,A,two
2,B,one
3,B,two
4,C,one
5,C,two


MultiIndex([('A', 'one'),
            ('A', 'two'),
            ('B', 'one'),
            ('B', 'two'),
            ('C', 'one'),
            ('C', 'two')],
           names=['first', 'second'])

In [24]:
df_4_frame = pd.DataFrame({'col_1':random_array1,
                           'col_2':random_array2}, 
                          index=multi_ind_frame)
df_4_frame

col_1  col_2
first second              
A     one        25     59
      two        81      8
B     one         3     70
      two        95     30
C     one         0     32
      two        35     70

## Пример № 1.7
## Метод .from_product() - Прямое (Декартово) произведение

In [25]:
level_0 = ['group_1', 'group_2', 'group_3']
level_1 = ['A', 'B']
multi_product=pd.MultiIndex.from_product([level_0, level_1],
                           names=['level_0', 'level_1'])
multi_product

MultiIndex([('group_1', 'A'),
            ('group_1', 'B'),
            ('group_2', 'A'),
            ('group_2', 'B'),
            ('group_3', 'A'),
            ('group_3', 'B')],
           names=['level_0', 'level_1'])

In [26]:
df_5_product = pd.DataFrame({'col_1':random_array1,
                             'col_2':random_array2}, 
                             index=multi_product)
df_5_product

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
group_2 A            3     70
        B           95     30
group_3 A            0     32
        B           35     70

## Пример № 1.8
## Метод .from_tuples() - из списка кортежей

In [27]:
tuples_list = [('group_1', 'A'), ('group_1', 'B'), ('group_1', 'C'), 
               ('group_2', 'A'), ('group_2', 'B'), ('group_2', 'C')]

multi_tuples = pd.MultiIndex.from_tuples(tuples_list, names=['level_0', 'level_1'])

multi_tuples

MultiIndex([('group_1', 'A'),
            ('group_1', 'B'),
            ('group_1', 'C'),
            ('group_2', 'A'),
            ('group_2', 'B'),
            ('group_2', 'C')],
           names=['level_0', 'level_1'])

In [28]:
df_6_tuples= pd.DataFrame({'col_1':random_array1,
                           'col_2':random_array2,}, 
                          index=multi_tuples)
df_6_tuples

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
        C            3     70
group_2 A           95     30
        B            0     32
        C           35     70

## Пример № 1.9  - с помощью аргументов levels и codes

    Параметры конструктора:
    
    levels - последовательность уникальных индексных меток для каждого уровня.
    codes - последовательность целых чисел для каждого уровня, обозначающие, какая метка - где находится.

In [29]:
midx = pd.MultiIndex(levels=[['group_1', 'group_2'], ['A', 'B', 'C']], 
                     codes=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]], 
                     names=['level_0', 'level_1'])
midx

MultiIndex([('group_1', 'A'),
            ('group_1', 'B'),
            ('group_1', 'C'),
            ('group_2', 'A'),
            ('group_2', 'B'),
            ('group_2', 'C')],
           names=['level_0', 'level_1'])

In [30]:
df_7 = pd.DataFrame({'col_1':random_array1,
                     'col_2':random_array2,}, 
                    index=midx)
df_7

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
        C            3     70
group_2 A           95     30
        B            0     32
        C           35     70

## Пример № 1.10 - передаем список массивов напрямую в Series или DataFrame

In [31]:
# Для автоматического создания MultiIndex можно передать список массивов непосредственно в Series или DataFrame:

mi_arrays = [np.array(['group_1', 'group_1', 'group_2', 'group_2', 'group_3', 'group_3', 'group_4', 'group_4']),
          np.array(['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B'])]

s = pd.Series(np.random.randint(0, 100, size=(8)), index=mi_arrays)
d = pd.DataFrame(np.random.randint(0, 100, size=(8,8)), index=mi_arrays, 
                 columns=['a','b','c','d','e','f','g','h'])

display(s, d)

group_1  A    82
         B    74
group_2  A    78
         B    88
group_3  A    26
         B    69
group_4  A    97
         B    44
dtype: int32

a   b   c   d   e   f   g   h
group_1 A  99  73  49  21   1  50  87  18
        B  24  19  93  12  63  74  31  52
group_2 A   1  10  55  28  64  71  62  90
        B  65  57  69  20  95  60  82  62
group_3 A  71  96  49  82  54  61  38  71
        B  25   5  82  32  60  40  89  50
group_4 A  13  63  84  37  24  20   2  83
        B  11  93  92  82  99  64  95  37

# Часть 2

# 2.1 Свойства и методы атрибутов index и columns. 

In [32]:
# создаем копию последнего результата, сохраняем в переменную df_mi
df_mi = df_7.copy()
df_mi

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
        C            3     70
group_2 A           95     30
        B            0     32
        C           35     70

## Атрибуты index и columns

In [33]:
print(df_mi.index, 
      df_mi.columns, sep='\n\n')

MultiIndex([('group_1', 'A'),
            ('group_1', 'B'),
            ('group_1', 'C'),
            ('group_2', 'A'),
            ('group_2', 'B'),
            ('group_2', 'C')],
           names=['level_0', 'level_1'])

Index(['col_1', 'col_2'], dtype='object')


## index.nlevels - Узнаем количество уровней

In [34]:
# количество уровней

print(df_mi.index.nlevels,
      df_mi.columns.nlevels, sep='\n')

2
1


## index.levshape - Узнаем длину каждого уровня

In [35]:
# кортеж с длиной каждого уровня

print(df_mi.index.levshape)

(2, 3)


In [36]:
#df_mi.columns.levshape
# AttributeError: 'Index' object has no attribute 'levshape'

## index.names -  Узнаем названия уровней

In [37]:
# FrozenList - специальная конструкция, которая используется для представления уровней, меток и имен объекта MultiIndex.

df_mi.index.names

FrozenList(['level_0', 'level_1'])

In [38]:
# можем изменить названия уровней, присвоив список, но лучше использовать метод index.set_names()

df_mi.index.names=['new_level_0', 'new_level_1']
df_mi

col_1  col_2
new_level_0 new_level_1              
group_1     A               25     59
            B               81      8
            C                3     70
group_2     A               95     30
            B                0     32
            C               35     70

In [39]:
df_mi.index.names

FrozenList(['new_level_0', 'new_level_1'])

## Метод index.set_names() - Смена имён уровней

In [40]:
display(df_mi.index.set_names(['lvl_0', 'lvl_1']),
        df_mi)

MultiIndex([('group_1', 'A'),
            ('group_1', 'B'),
            ('group_1', 'C'),
            ('group_2', 'A'),
            ('group_2', 'B'),
            ('group_2', 'C')],
           names=['lvl_0', 'lvl_1'])

col_1  col_2
new_level_0 new_level_1              
group_1     A               25     59
            B               81      8
            C                3     70
group_2     A               95     30
            B                0     32
            C               35     70

In [41]:
# изменяем исходный объект
df_mi.index.set_names(['lvl_0', 'lvl_1'], inplace=True)
df_mi

col_1  col_2
lvl_0   lvl_1              
group_1 A         25     59
        B         81      8
        C          3     70
group_2 A         95     30
        B          0     32
        C         35     70

In [42]:
df_mi_2 = df_mi.copy() # создаем копию df_mi
# можно поменять название конкретного уровня
# если хотим изменить исходный DataFrame, указываем inplace=True
df_mi_2.index.set_names('LEVEL_1', level=1, inplace=True)
df_mi_2

col_1  col_2
lvl_0   LEVEL_1              
group_1 A           25     59
        B           81      8
        C            3     70
group_2 A           95     30
        B            0     32
        C           35     70

## Метод index.get_level_values() - Просмотр меток уровней MultiIndex

In [43]:
# принимает уровень в виде числа
df_mi.index.get_level_values(0)

Index(['group_1', 'group_1', 'group_1', 'group_2', 'group_2', 'group_2'], dtype='object', name='lvl_0')

In [44]:
# или в виде индексной метки
df_mi.index.get_level_values('lvl_0')

Index(['group_1', 'group_1', 'group_1', 'group_2', 'group_2', 'group_2'], dtype='object', name='lvl_0')

## index.levels - Получаем уникальные элементы, содержащиеся в MultiIndex

In [45]:
# элементы, содержащиеся в мультииндексе

df_mi.index.levels

FrozenList([['group_1', 'group_2'], ['A', 'B', 'C']])

## Передача нового мультииндекса через атрибут index или columns

In [46]:
# создаем новый мультииндекс 

tuples_list_rus = [('группа_1', 'A'), ('группа_1', 'B'), ('группа_1', 'C'), 
               ('группа_2', 'A'), ('группа_2', 'B'), ('группа_2', 'C')]

multi_tuples_rus = pd.MultiIndex.from_tuples(tuples_list_rus, names=['уровень_0', 'уровень_1'])

multi_tuples_rus

MultiIndex([('группа_1', 'A'),
            ('группа_1', 'B'),
            ('группа_1', 'C'),
            ('группа_2', 'A'),
            ('группа_2', 'B'),
            ('группа_2', 'C')],
           names=['уровень_0', 'уровень_1'])

In [47]:
# передаем новый мультииндекс через атрибут index

df_mi.index = multi_tuples_rus
df_mi

col_1  col_2
уровень_0 уровень_1              
группа_1  A             25     59
          B             81      8
          C              3     70
группа_2  A             95     30
          B              0     32
          C             35     70

In [48]:
# возвращаем старый мультииндекс

df_mi.index=multi_tuples
df_mi

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
        C            3     70
group_2 A           95     30
        B            0     32
        C           35     70

## Метод set_levels() - Изменение значений индексов на конкретных уровнях

In [49]:
#  смотрим на многоуровневый индекс и на его уровни (уникальные элементы уровней)

display(df_mi.index,
        df_mi.index.levels)

MultiIndex([('group_1', 'A'),
            ('group_1', 'B'),
            ('group_1', 'C'),
            ('group_2', 'A'),
            ('group_2', 'B'),
            ('group_2', 'C')],
           names=['level_0', 'level_1'])

FrozenList([['group_1', 'group_2'], ['A', 'B', 'C']])

In [50]:
# Метод set_levels не меняет исходный мультииндекс, а возвращает новый
# количество элементов в новых списках либо больше либо равно количеству элементов в соответствующих им уровнях

new_index = df_mi.index.set_levels([['g1', 'g2', 'g3'], ['a', 'b', 'c', 'd']])

display(new_index,
        # если мы посмотрим свойство levels, то увидим все, даже неиспользуемые элементы уровней индекса
        new_index.levels)

MultiIndex([('g1', 'a'),
            ('g1', 'b'),
            ('g1', 'c'),
            ('g2', 'a'),
            ('g2', 'b'),
            ('g2', 'c')],
           names=['level_0', 'level_1'])

FrozenList([['g1', 'g2', 'g3'], ['a', 'b', 'c', 'd']])

In [51]:
# можем указать другой порядок уровней для установки новых значений

new_index_2 = df_mi.index.set_levels([['g1', 'g2', 'g3'], ['a', 'b', 'c', 'd']], level=[1, 0])

display(new_index_2,
        # если мы посмотрим свойство levels, то увидим все, даже неиспользуемые элементы уровней индекса
        new_index_2.levels)

MultiIndex([('a', 'g1'),
            ('a', 'g2'),
            ('a', 'g3'),
            ('b', 'g1'),
            ('b', 'g2'),
            ('b', 'g3')],
           names=['level_0', 'level_1'])

FrozenList([['a', 'b', 'c', 'd'], ['g1', 'g2', 'g3']])

In [52]:
# можем указать только один уровень, не меняя остальные

new_index_3 = df_mi.index.set_levels(['g1', 'g2', 'g3'], level=0)

display(new_index_3,
        # если мы посмотрим свойство levels, то увидим все, даже неиспользуемые элементы уровней индекса
        new_index_3.levels)

MultiIndex([('g1', 'A'),
            ('g1', 'B'),
            ('g1', 'C'),
            ('g2', 'A'),
            ('g2', 'B'),
            ('g2', 'C')],
           names=['level_0', 'level_1'])

FrozenList([['g1', 'g2', 'g3'], ['A', 'B', 'C']])

## Удаление уровней

## Метод index.remove_unused_levels() - Удаление неиспользуемых меток уровней

In [53]:
display(new_index_3, 
        new_index_3.levels)

MultiIndex([('g1', 'A'),
            ('g1', 'B'),
            ('g1', 'C'),
            ('g2', 'A'),
            ('g2', 'B'),
            ('g2', 'C')],
           names=['level_0', 'level_1'])

FrozenList([['g1', 'g2', 'g3'], ['A', 'B', 'C']])

In [54]:
index_r_l = new_index_3.remove_unused_levels()
display(index_r_l, 
        index_r_l.levels)

MultiIndex([('g1', 'A'),
            ('g1', 'B'),
            ('g1', 'C'),
            ('g2', 'A'),
            ('g2', 'B'),
            ('g2', 'C')],
           names=['level_0', 'level_1'])

FrozenList([['g1', 'g2'], ['A', 'B', 'C']])

## Метод index.droplevel() - Удаление конкретного уровня

In [55]:
display(df_mi, 
        df_mi.index)

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
        C            3     70
group_2 A           95     30
        B            0     32
        C           35     70

MultiIndex([('group_1', 'A'),
            ('group_1', 'B'),
            ('group_1', 'C'),
            ('group_2', 'A'),
            ('group_2', 'B'),
            ('group_2', 'C')],
           names=['level_0', 'level_1'])

In [56]:
df_mi.index.droplevel()

Index(['A', 'B', 'C', 'A', 'B', 'C'], dtype='object', name='level_1')

In [57]:
# прописываем целочисленную позицию уровня для удаления
df_mi.index.droplevel(level=1)

Index(['group_1', 'group_1', 'group_1', 'group_2', 'group_2', 'group_2'], dtype='object', name='level_0')

In [58]:
# прописываем имя уровня для удаления
df_mi.index.droplevel(['level_1'])

Index(['group_1', 'group_1', 'group_1', 'group_2', 'group_2', 'group_2'], dtype='object', name='level_0')

#### Все уровни удалять нельзя! Иначе pandas вернет ошибку.

# 2.2 Методы структур Series и DataFrame  для работы с MultiIndex

## DataFrame.set_index() - Установка MultiIndex

In [59]:
df_si = pd.DataFrame({'col_0*': [0, 0, 1, 1], 'col_1*': ['x', 'x', 'z', 'y'], 
                      'col_2': np.random.randint(0, 100, size=(4)), 'col_3': np.random.randint(0, 100, size=(4))})

# используем устанавливаем существующие колонки DataFrame в качестве уровней иерархического индекса
# чтобы изменения произошли в изначальном объекте DataFrame нужно использовать параметр inplace=True
display(df_si,
        df_si.set_index(['col_0*', 'col_1*']))

,col_0*,col_1*,col_2,col_3
0,0,x,27,19
1,0,x,32,55
2,1,z,0,8
3,1,y,93,21


col_2  col_3
col_0* col_1*              
0      x          27     19
       x          32     55
1      z           0      8
       y          93     21

In [60]:
# можем также передать новую последовательность индексных меток в конструкторе pd.Index()

df_si.set_index([pd.Index(['row_1', 'row_2', 'row_3', 'row_4'], name='rows'), 'col_0*', 'col_1*'], inplace=True)
display(df_si,
        df_si.index.names,
        df_si.index.levels)

,,,col_2,col_3
rows,col_0*,col_1*,,
row_1,0,x,27,19
row_2,0,x,32,55
row_3,1,z,0,8
row_4,1,y,93,21


FrozenList(['rows', 'col_0*', 'col_1*'])

FrozenList([['row_1', 'row_2', 'row_3', 'row_4'], [0, 1], ['x', 'y', 'z']])

## DataFrame.reset_index() - Сброс индекса (уровней MultiIndex)

In [61]:
df_mi_3 = df_mi.copy()
df_mi_3

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
        C            3     70
group_2 A           95     30
        B            0     32
        C           35     70

In [62]:
# по умолчанию сбрасываем все уровни, остается RangeIndex
# не изменяем объект DataFrame, а получаем новый

df_mi_3.reset_index()

,level_0,level_1,col_1,col_2
0,group_1,A,25,59
1,group_1,B,81,8
2,group_1,C,3,70
3,group_2,A,95,30
4,group_2,B,0,32
5,group_2,C,35,70


In [63]:
# сбрасываем только первый уровень
# сохраняем изменения в изначальном объекте 

df_mi_3.reset_index(['level_1'],inplace=True)
df_mi_3

,level_1,col_1,col_2
level_0,,,
group_1,A,25,59
group_1,B,81,8
group_1,C,3,70
group_2,A,95,30
group_2,B,0,32
group_2,C,35,70


## DataFrame.swaplevel() - Смена уровней

In [64]:
# переключаем порядок двух уровней
display(df_mi,
        df_mi.swaplevel())

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
        C            3     70
group_2 A           95     30
        B            0     32
        C           35     70

,,col_1,col_2
level_1,level_0,,
A,group_1,25,59
B,group_1,81,8
C,group_1,3,70
A,group_2,95,30
B,group_2,0,32
C,group_2,35,70


In [65]:
agg_result

data1       data2      
            sum  mean   sum  mean
key1 key2                        
A    no       3  3.00     9  9.00
     yes      0  0.00     8  8.00
B    no      20  6.67    19  6.33
     yes      8  4.00     3  1.50
C    no      19  6.33    11  3.67
     yes     16  8.00    14  7.00

In [66]:
# agg_result.swaplevel(0, 1, axis=0)
agg_result.swaplevel(0, 1)

data1       data2      
            sum  mean   sum  mean
key2 key1                        
no   A        3  3.00     9  9.00
yes  A        0  0.00     8  8.00
no   B       20  6.67    19  6.33
yes  B        8  4.00     3  1.50
no   C       19  6.33    11  3.67
yes  C       16  8.00    14  7.00

In [67]:
agg_result.swaplevel(0, 1, axis=1)

sum  mean   sum  mean
          data1 data1 data2 data2
key1 key2                        
A    no       3  3.00     9  9.00
     yes      0  0.00     8  8.00
B    no      20  6.67    19  6.33
     yes      8  4.00     3  1.50
C    no      19  6.33    11  3.67
     yes     16  8.00    14  7.00

## DataFrame.reorder_levels() - Изменение порядка уровней 

In [68]:
# создаем DataFrame с 4 уровнями индекса строк

arrays_4_levels=[['group_1','group_1','group_2','group_2'],['A','B','C','D'], 
          ['yes','no','yes','no'],[2,1,2,1]]
index_4_levels= pd.MultiIndex.from_arrays(arrays_4_levels, names=['level_0', 'level_1', 'level_2', 'level_3'])
print(index_4_levels)

df_4_levels = pd.DataFrame(random_matrix, index=index_4_levels)
display(df_4_levels)

MultiIndex([('group_1', 'A', 'yes', 2),
            ('group_1', 'B',  'no', 1),
            ('group_2', 'C', 'yes', 2),
            ('group_2', 'D',  'no', 1)],
           names=['level_0', 'level_1', 'level_2', 'level_3'])


0   1   2   3
level_0 level_1 level_2 level_3                
group_1 A       yes     2        51  78  71  16
        B       no      1        67  43  81  57
group_2 C       yes     2        79  42  67  20
        D       no      1        90  29  91  52

In [69]:
# df_4_levels.reorder_levels([2, 0, 3, 1], axis=0)
# переставляем уровни MultiIndex за один шаг

df_4_levels.reorder_levels([2, 0, 3, 1])

,,,,0,1,2,3
level_2,level_0,level_3,level_1,,,,
yes,group_1,2,A,51,78,71,16
no,group_1,1,B,67,43,81,57
yes,group_2,2,C,79,42,67,20
no,group_2,1,D,90,29,91,52


## DataFrame.reindex() -  Использование переиндексации MultiIndex

In [70]:
display(df_mi, 
        df_mi.index)

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
        C            3     70
group_2 A           95     30
        B            0     32
        C           35     70

MultiIndex([('group_1', 'A'),
            ('group_1', 'B'),
            ('group_1', 'C'),
            ('group_2', 'A'),
            ('group_2', 'B'),
            ('group_2', 'C')],
           names=['level_0', 'level_1'])

In [71]:
# используем срез для переиндексации

df_mi.reindex(df_mi.index[:3])

col_1  col_2
level_0 level_1              
group_1 A           25     59
        B           81      8
        C            3     70

In [72]:
# используем массив кортежей с абсолютно новыми индексными парами
df_mi.reindex([('group_one','a'), ('group_one','b'), ('group_one','c'), 
               ('group_two','a'), ('group_two','b'), ('group_two','c')])

col_1  col_2
level_0   level_1              
group_one a          NaN    NaN
          b          NaN    NaN
          c          NaN    NaN
group_two a          NaN    NaN
          b          NaN    NaN
          c          NaN    NaN

In [73]:
# используем массив кортежей с новыми и старыми индексными парами
df_mi.reindex([('group_1','A'), ('group_1','b'), ('group_1','C'), 
               ('group_two','a'), ('group_two','b'), ('group_two','c')])

col_1  col_2
level_0   level_1              
group_1   A         25.0   59.0
          b          NaN    NaN
          C          3.0   70.0
group_two a          NaN    NaN
          b          NaN    NaN
          c          NaN    NaN

## DataFrame.rename() - Переименование меток Index или MultiIndex

In [74]:
df_4_levels

0   1   2   3
level_0 level_1 level_2 level_3                
group_1 A       yes     2        51  78  71  16
        B       no      1        67  43  81  57
group_2 C       yes     2        79  42  67  20
        D       no      1        90  29  91  52

In [75]:
# чтобы изменить метки индексов строк, передаем mapper (словарь замен) в параметр index
df_4_levels.rename(index={'group_1': 'группа_1', 'group_2': 'группа_2', 
                          'A': 'А', 'B': 'Б', 'C': 'В', 'D': 'Г',
                         'yes': 'да', 'no': 'нет'})

0   1   2   3
level_0  level_1 level_2 level_3                
группа_1 А       да      2        51  78  71  16
         Б       нет     1        67  43  81  57
группа_2 В       да      2        79  42  67  20
         Г       нет     1        90  29  91  52

In [76]:
# можем передать просто mapper (словарь замен) без параметра index
# по умолчанию изменяться будут метки индексов строк

df_4_levels.rename({'group_1': 'группа_1', 'group_2': 'группа_2', 
                          'A': 'А', 'B': 'Б', 'C': 'В', 'D': 'Г',
                         'yes': 'да', 'no': 'нет'})

0   1   2   3
level_0  level_1 level_2 level_3                
группа_1 А       да      2        51  78  71  16
         Б       нет     1        67  43  81  57
группа_2 В       да      2        79  42  67  20
         Г       нет     1        90  29  91  52

In [77]:
# чтобы изменить метки индексов столбцов передаем mapper (словарь замен) в параметр columns

df_4_levels.rename(columns={0: 'col_0', 1: 'col_1', 2: 'col_2', 3: 'col_3'})

col_0  col_1  col_2  col_3
level_0 level_1 level_2 level_3                            
group_1 A       yes     2           51     78     71     16
        B       no      1           67     43     81     57
group_2 C       yes     2           79     42     67     20
        D       no      1           90     29     91     52

In [78]:
# можем использовать axis=1 для указания оси столбцов

df_4_levels.rename({0: 'col_0', 1: 'col_1', 2: 'col_2', 3: 'col_3'}, axis=1)

col_0  col_1  col_2  col_3
level_0 level_1 level_2 level_3                            
group_1 A       yes     2           51     78     71     16
        B       no      1           67     43     81     57
group_2 C       yes     2           79     42     67     20
        D       no      1           90     29     91     52

In [79]:
# если хотим изменить метки индексов столбцов, то обязательно нужно указать columns или axis=1
# df_4_levels.rename({0: 'col_0', 1: 'col_1', 2: 'col_2', 3: 'col_3'}, axis=1)
# df_4_levels.rename(columns={0: 'col_0', 1: 'col_1', 2: 'col_2', 3: 'col_3'})

# в противном случае, получим изменения в метках индексов строк
df_4_levels.rename({0: 'col_0', 1: 'col_1', 2: 'col_2', 3: 'col_3'})

0   1   2   3
level_0 level_1 level_2 level_3                
group_1 A       yes     col_2    51  78  71  16
        B       no      col_1    67  43  81  57
group_2 C       yes     col_2    79  42  67  20
        D       no      col_1    90  29  91  52

## DataFrame.rename_axis() - Переименование уровней MultiIndex (имени Index)

    Ось 0 - имя индекса (имена уровней индекса) строк

In [80]:
# можно передать словарь замен (mapper) в 
df_4_levels.rename_axis(index={'level_0': 'уровень_0', 
                               'level_1': 'уровень_1', 
                               'level_2': 'уровень_2', 
                               'level_3': 'уровень_3'})

0   1   2   3
уровень_0 уровень_1 уровень_2 уровень_3                
group_1   A         yes       2          51  78  71  16
          B         no        1          67  43  81  57
group_2   C         yes       2          79  42  67  20
          D         no        1          90  29  91  52

In [81]:
# Если используем mapper, обязательно нужно указывать конкретную ось параметром index или columns! Иначе получим ошибку!
# df_4_levels.rename_axis({'level_0': 'уровень_0', 'level_1': 'уровень_1', 'level_2': 'уровень_2', 'level_3': 'уровень_3'})

In [82]:
# можно передать список соответсвующей длины

df_4_levels.rename_axis(['L_0', 'L_1', 'L_2', 'L_3'])

0   1   2   3
L_0     L_1 L_2 L_3                
group_1 A   yes 2    51  78  71  16
        B   no  1    67  43  81  57
group_2 C   yes 2    79  42  67  20
        D   no  1    90  29  91  52

In [83]:
# можно передать функцию

df_4_levels.rename_axis(index=str.upper)

0   1   2   3
LEVEL_0 LEVEL_1 LEVEL_2 LEVEL_3                
group_1 A       yes     2        51  78  71  16
        B       no      1        67  43  81  57
group_2 C       yes     2        79  42  67  20
        D       no      1        90  29  91  52

    Ось 1 - имя индекса (имена уровней индекса) столбцов

In [84]:
# использование DataFrame.rename_axis() с аргументом columns изменит имя индекса столбцов (имя оси столбцов)
# можно дать имя оси columns ( прописать axis='columns' или axis=1)

display(df_4_levels.rename_axis(columns='столбцы'),
        df_4_levels.rename_axis('столбцы', axis='columns'),
        df_4_levels.rename_axis('столбцы', axis=1),
        df_4_levels.rename_axis(columns='столбцы').columns)

столбцы                           0   1   2   3
level_0 level_1 level_2 level_3                
group_1 A       yes     2        51  78  71  16
        B       no      1        67  43  81  57
group_2 C       yes     2        79  42  67  20
        D       no      1        90  29  91  52

столбцы                           0   1   2   3
level_0 level_1 level_2 level_3                
group_1 A       yes     2        51  78  71  16
        B       no      1        67  43  81  57
group_2 C       yes     2        79  42  67  20
        D       no      1        90  29  91  52

столбцы                           0   1   2   3
level_0 level_1 level_2 level_3                
group_1 A       yes     2        51  78  71  16
        B       no      1        67  43  81  57
group_2 C       yes     2        79  42  67  20
        D       no      1        90  29  91  52

RangeIndex(start=0, stop=4, step=1, name='столбцы')

## DataFrame.sort_index() - Сортировка уровней MultiIndex (Index)

In [85]:
# создаем два списка из двух массивов numpy одинаковой длины каждый
mi_arrays_2 = [np.array(['группа_3', 'группа_3', 'группа_2', 'группа_2', 'группа_1', 'группа_1', 'группа_1', 
                       'группа_5', 'группа_5', 'группа_5', 'группа_4', 'группа_4']),
          np.array(['Орлов А.А.', 'Иванов Н.Н.', 'Петров П.А.', 'Смирнов А.Г.', 'Кузнецов Н.А.', 
                    'Александров П.П.', 'Соловьев Л.В.', 'Григорьев В.В.', 'Раевский Д.А.', 
                    'Быков Л.Р.', 'Холодов Н.Ш.', 'Ногинцев А.К.'])]

mi_arrays_3 = [np.array([2023, 2023, 2023, 2023, 2022, 2022, 2022, 2022]),
          np.array(['1_кв.', '2_кв.', '3_кв.', '4_кв.', '1_кв.', '2_кв.', '3_кв.', '4_кв.'])]

# создаем DataFrame с мультииндексами строк и столбцов
df_q = pd.DataFrame(np.random.randint(30, 100, size=(12, 8)), index=mi_arrays_2, 
                    columns=mi_arrays_3)

# Устанавливаем названия уровней индекса строк
df_q.index.set_names(['ГРУППА', 'ФИО СОТРУДНИКА'], inplace=True)
df_q.columns.set_names(['год', 'квартал'], inplace=True)
df_q

год                        2023                    2022                  
квартал                   1_кв. 2_кв. 3_кв. 4_кв. 1_кв. 2_кв. 3_кв. 4_кв.
ГРУППА   ФИО СОТРУДНИКА                                                  
группа_3 Орлов А.А.          55    55    38    64    52    45    64    39
         Иванов Н.Н.         56    45    55    79    50    83    48    46
группа_2 Петров П.А.         93    63    52    69    72    82    74    67
         Смирнов А.Г.        81    76    51    87    74    85    69    48
группа_1 Кузнецов Н.А.       32    73    89    48    84    89    58    40
         Александров П.П.    50    71    61    87    64    32    65    95
         Соловьев Л.В.       82    72    74    92    69    95    94    80
группа_5 Григорьев В.В.      92    79    54    35    98    31    65    80
         Раевский Д.А.       78    51    77    37    63    71    80    95
         Быков Л.Р.          38    88    67    45    47    82    65    76
группа_4 Холодов Н.Ш.        55    87    71    79    97    70    52    92
         Ногинцев А.К.       36    54    96    85    91    34    62    53

In [86]:
# производим сортировку, по умолчанию сортировка производится по строкам
# чтобы отсортировать и сохранить результат используем inplace=True: df_q.sort_index(inplace=True)

df_q.sort_index()

год                        2023                    2022                  
квартал                   1_кв. 2_кв. 3_кв. 4_кв. 1_кв. 2_кв. 3_кв. 4_кв.
ГРУППА   ФИО СОТРУДНИКА                                                  
группа_1 Александров П.П.    50    71    61    87    64    32    65    95
         Кузнецов Н.А.       32    73    89    48    84    89    58    40
         Соловьев Л.В.       82    72    74    92    69    95    94    80
группа_2 Петров П.А.         93    63    52    69    72    82    74    67
         Смирнов А.Г.        81    76    51    87    74    85    69    48
группа_3 Иванов Н.Н.         56    45    55    79    50    83    48    46
         Орлов А.А.          55    55    38    64    52    45    64    39
группа_4 Ногинцев А.К.       36    54    96    85    91    34    62    53
         Холодов Н.Ш.        55    87    71    79    97    70    52    92
группа_5 Быков Л.Р.          38    88    67    45    47    82    65    76
         Григорьев В.В.      92    79    54    35    98    31    65    80
         Раевский Д.А.       78    51    77    37    63    71    80    95

In [87]:
# если хотим отсортировать индексы строк по убыванию
# по умолчанию ascending=True

df_q.sort_index(ascending=False)

год                        2023                    2022                  
квартал                   1_кв. 2_кв. 3_кв. 4_кв. 1_кв. 2_кв. 3_кв. 4_кв.
ГРУППА   ФИО СОТРУДНИКА                                                  
группа_5 Раевский Д.А.       78    51    77    37    63    71    80    95
         Григорьев В.В.      92    79    54    35    98    31    65    80
         Быков Л.Р.          38    88    67    45    47    82    65    76
группа_4 Холодов Н.Ш.        55    87    71    79    97    70    52    92
         Ногинцев А.К.       36    54    96    85    91    34    62    53
группа_3 Орлов А.А.          55    55    38    64    52    45    64    39
         Иванов Н.Н.         56    45    55    79    50    83    48    46
группа_2 Смирнов А.Г.        81    76    51    87    74    85    69    48
         Петров П.А.         93    63    52    69    72    82    74    67
группа_1 Соловьев Л.В.       82    72    74    92    69    95    94    80
         Кузнецов Н.А.       32    73    89    48    84    89    58    40
         Александров П.П.    50    71    61    87    64    32    65    95

In [88]:
# производим сортировку по году 

df_q.sort_index(level=0, axis=1)

# можем написать level='год' вместо level=0 : 
# df_q.sort_index(level='год', axis=1))

год                        2022                    2023                  
квартал                   1_кв. 2_кв. 3_кв. 4_кв. 1_кв. 2_кв. 3_кв. 4_кв.
ГРУППА   ФИО СОТРУДНИКА                                                  
группа_3 Орлов А.А.          52    45    64    39    55    55    38    64
         Иванов Н.Н.         50    83    48    46    56    45    55    79
группа_2 Петров П.А.         72    82    74    67    93    63    52    69
         Смирнов А.Г.        74    85    69    48    81    76    51    87
группа_1 Кузнецов Н.А.       84    89    58    40    32    73    89    48
         Александров П.П.    64    32    65    95    50    71    61    87
         Соловьев Л.В.       69    95    94    80    82    72    74    92
группа_5 Григорьев В.В.      98    31    65    80    92    79    54    35
         Раевский Д.А.       63    71    80    95    78    51    77    37
         Быков Л.Р.          47    82    65    76    38    88    67    45
группа_4 Холодов Н.Ш.        97    70    52    92    55    87    71    79
         Ногинцев А.К.       91    34    62    53    36    54    96    85

In [89]:
# производим сотрировку по кварталу (сначала 1 кв. каждого года, потом 2-й кв. и так далее)

df_q.sort_index(level='квартал', axis=1)

год                        2022  2023  2022  2023  2022  2023  2022  2023
квартал                   1_кв. 1_кв. 2_кв. 2_кв. 3_кв. 3_кв. 4_кв. 4_кв.
ГРУППА   ФИО СОТРУДНИКА                                                  
группа_3 Орлов А.А.          52    55    45    55    64    38    39    64
         Иванов Н.Н.         50    56    83    45    48    55    46    79
группа_2 Петров П.А.         72    93    82    63    74    52    67    69
         Смирнов А.Г.        74    81    85    76    69    51    48    87
группа_1 Кузнецов Н.А.       84    32    89    73    58    89    40    48
         Александров П.П.    64    50    32    71    65    61    95    87
         Соловьев Л.В.       69    82    95    72    94    74    80    92
группа_5 Григорьев В.В.      98    92    31    79    65    54    80    35
         Раевский Д.А.       63    78    71    51    80    77    95    37
         Быков Л.Р.          47    38    82    88    65    67    76    45
группа_4 Холодов Н.Ш.        97    55    70    87    52    71    92    79
         Ногинцев А.К.       91    36    34    54    62    96    53    85

In [90]:
# производим поочередную сотрировку с сохранением результата в изначальной структуре DataFrame

df_q.sort_index(inplace=True)
df_q.sort_index(axis=1, inplace=True)
df_q

год                        2022                    2023                  
квартал                   1_кв. 2_кв. 3_кв. 4_кв. 1_кв. 2_кв. 3_кв. 4_кв.
ГРУППА   ФИО СОТРУДНИКА                                                  
группа_1 Александров П.П.    64    32    65    95    50    71    61    87
         Кузнецов Н.А.       84    89    58    40    32    73    89    48
         Соловьев Л.В.       69    95    94    80    82    72    74    92
группа_2 Петров П.А.         72    82    74    67    93    63    52    69
         Смирнов А.Г.        74    85    69    48    81    76    51    87
группа_3 Иванов Н.Н.         50    83    48    46    56    45    55    79
         Орлов А.А.          52    45    64    39    55    55    38    64
группа_4 Ногинцев А.К.       91    34    62    53    36    54    96    85
         Холодов Н.Ш.        97    70    52    92    55    87    71    79
группа_5 Быков Л.Р.          47    82    65    76    38    88    67    45
         Григорьев В.В.      98    31    65    80    92    79    54    35
         Раевский Д.А.       63    71    80    95    78    51    77    37

## Полезные свойства

In [91]:
display(df_q.index, df_q.columns)

MultiIndex([('группа_1', 'Александров П.П.'),
            ('группа_1',    'Кузнецов Н.А.'),
            ('группа_1',    'Соловьев Л.В.'),
            ('группа_2',      'Петров П.А.'),
            ('группа_2',     'Смирнов А.Г.'),
            ('группа_3',      'Иванов Н.Н.'),
            ('группа_3',       'Орлов А.А.'),
            ('группа_4',    'Ногинцев А.К.'),
            ('группа_4',     'Холодов Н.Ш.'),
            ('группа_5',       'Быков Л.Р.'),
            ('группа_5',   'Григорьев В.В.'),
            ('группа_5',    'Раевский Д.А.')],
           names=['ГРУППА', 'ФИО СОТРУДНИКА'])

MultiIndex([(2022, '1_кв.'),
            (2022, '2_кв.'),
            (2022, '3_кв.'),
            (2022, '4_кв.'),
            (2023, '1_кв.'),
            (2023, '2_кв.'),
            (2023, '3_кв.'),
            (2023, '4_кв.')],
           names=['год', 'квартал'])

In [92]:
print(df_q.index.is_monotonic_increasing, # отсортирован ли индекс(мультииндекс) строк по возрастанию (ДА)
      df_q.columns.is_monotonic_increasing,# отсортирован ли индекс(мультииндекс) столбцов по возрастанию (ДА)
      df_q.index.is_monotonic_decreasing, # отсортирован ли индекс(мультииндекс) строк по убыванию (НЕТ)
      df_q.columns.is_monotonic_decreasing,# отсортирован ли индекс(мультииндекс) столбцов по убыванию (НЕТ)
      df_q.index.has_duplicates,# есть ли в индексе(мультииндексе) строк дубликаты (НЕТ)
      df_q.columns.has_duplicates,# есть ли в индексе(мультииндексе) столбцов дубликаты (НЕТ)
      df_q.index.is_unique, # уникален ли индекс(мультииндекс) строк (ДА)
      df_q.columns.is_unique, # уникален ли индекс(мультииндекс) столбцов (ДА)
      sep='\n')

True
True
False
False
False
False
True
True
